# Description

This notebook will compute TSNE for the multi-task dataset. For UMAP we explore three hyper-parameters:

* Distance Function: euclidean, cosine or correlation
* knn: neighborhood size
* m: final number of dimensions
* learning rate: for the optimization phase

Matrices will be written as pandas pickle objects in ```/data/SFIMJGC_HCP7T/manifold_learning/Data_Interim/PNAS2015/{sbj}/UMAP```

In [1]:
import pandas as pd
import numpy as np
import os
import os.path as osp
import getpass
from datetime import datetime
from utils.basics import PNAS2015_subject_list, PNAS2015_folder, PRJ_DIR
from utils.basics import tsne_dist_metrics, tsne_pps, tsne_ms, tsne_alphas, tsne_inits
from utils.basics import input_datas, norm_methods

***

The next cell select the Window Length ```wls``` and Window Step ```wss``` used to generate the matrices

In [2]:
wls      = 45
wss      = 1.5

***
# 1. Compute T-SNE Scan Level Embeddings

## 1.1. Compute TSNE Embeddings on all input types
Those are the scenarios we will be running

In [3]:
print('++ INFO: Distance Metrics: %s' % str(tsne_dist_metrics))
print('++ INFO: Perplexitiess:    %s' % str(tsne_pps))
print('++ INFO: Ms:               %s' % str(tsne_ms))
print('++ INFO: Learning Rates:   %s' % str(tsne_alphas))
print('++ INFO: Init Methods:     %s' % str(tsne_inits))

++ INFO: Distance Metrics: ['euclidean', 'correlation', 'cosine']
++ INFO: Perplexitiess:    [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 125, 150, 175, 200]
++ INFO: Ms:               [2, 3, 5, 10, 15, 20, 25, 30]
++ INFO: Learning Rates:   [10, 50, 75, 100, 200, 500, 1000]
++ INFO: Init Methods:     ['pca']


The next cell will create the output folders if they do not exist already

In [4]:
# Create Output Folders if they do not exists
for subject in PNAS2015_subject_list:
    for input_data in input_datas:
        path = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',subject,'TSNE', input_data)
        if not osp.exists(path):
            print('++ INFO: Creating folder [%s]' % path)
            os.makedirs(path)

The next cell will create folders for the swarm log files and for the actual swarm script. Those folders are created using the username as part of their name. That way it is easier for different users to work together on the project.

In [24]:
#user specific folders
#=====================
username = getpass.getuser()
print('++ INFO: user working now --> %s' % username)

swarm_folder   = osp.join(PRJ_DIR,'SwarmFiles.{username}'.format(username=username))
logs_folder    = osp.join(PRJ_DIR,'Logs.{username}'.format(username=username))  

swarm_path     = osp.join(swarm_folder,'N08_TSNE_Multitask_Scans.SWARM.sh')
logdir_path    = osp.join(logs_folder, 'N08_TSNE_Multitask_Scans.logs')

if not osp.exists(swarm_folder):
    os.makedirs(swarm_folder)
if not osp.exists(logdir_path):
    os.makedirs(logdir_path)
print('++ INFO: Swarm File  : %s' % swarm_path)
print('++ INFO: Logs Folder : %s' % logdir_path)

++ INFO: user working now --> javiergc
++ INFO: Swarm File  : /data/SFIMJGC_HCP7T/manifold_learning_fmri/SwarmFiles.javiergc/N08_TSNE_Multitask_Scans.SWARM.sh
++ INFO: Logs Folder : /data/SFIMJGC_HCP7T/manifold_learning_fmri/Logs.javiergc/N08_TSNE_Multitask_Scans.logs


Create swarm script. This script will have one line per matrix to be generated.

> NOTE: For the group level, we will work on extra dimensions (becuase of Procrustes) but only look at Original Data, correlation metric and 10,1000 as learning rate.

In [26]:
%%time
# Open the file
n_jobs=16
swarm_file = open(swarm_path, "w")
# Log the date and time when the SWARM file is created
swarm_file.write('#Create Time: %s' % datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
swarm_file.write('\n')

# Insert comment line with SWARM command
swarm_file.write('#swarm -J TSNE_10_Scan -f {swarm_path} -b 15 -g 16 -t {n_jobs} --time 00:16:00 --partition quick,norm --logdir {logdir_path}'.format(swarm_path=swarm_path,logdir_path=logdir_path, n_jobs=n_jobs))
swarm_file.write('\n')
num_entries = 0 
num_iters = 0

for input_data in input_datas:
    for subject in PNAS2015_subject_list:
        for norm_method in norm_methods:
            for dist in tsne_dist_metrics:
                for init_method in tsne_inits:
                    for pp in tsne_pps:
                        for alpha in tsne_alphas:
                            for m in [15]:#tsne_ms:
                                num_iters += 1
                                path_tvfc = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',subject,input_data,       '{subject}_Craddock_0200.WL{wls}s.WS{wss}s.tvFC.Z.{nm}.pkl'.format(subject=subject,nm=norm_method,wls=str(int(wls)).zfill(3), wss=str(wss)))
                                path_out  = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',subject,'TSNE',input_data,'{subject}_Craddock_0200.WL{wls}s.WS{wss}s.TSNE_{dist}_pp{pp}_m{m}_a{lr}_{init_method}.{nm}.pkl'.format(subject=subject,
                                                                                                                                                   nm = norm_method,
                                                                                                                                                   wls=str(int(wls)).zfill(3), 
                                                                                                                                                   wss=str(wss),
                                                                                                                                                   init_method=init_method,
                                                                                                                                                   dist=dist,
                                                                                                                                                   pp=str(pp).zfill(4),
                                                                                                                                                   m=str(m).zfill(4),
                                                                                                                                                   lr=str(alpha)))
                                if not osp.exists(path_out):
                                    num_entries += 1
                                    swarm_file.write('export path_tvfc={path_tvfc} dist={dist} pp={pp} lr={lr} m={m} n_iter=10000 init={init_method} path_out={path_out} n_jobs={n_jobs} norm={norm_method} grad_method=exact; sh {scripts_dir}/N08_TSNE.sh'.format(path_tvfc=path_tvfc, 
                                                                                                                                    path_out=path_out,
                                                                                                                                    dist=dist,
                                                                                                                                    init_method=init_method,
                                                                                                                                    norm_method=norm_method,
                                                                                                                                    pp=pp,
                                                                                                                                    m=m, 
                                                                                                                                    lr=alpha,
                                                                                                                                    n_jobs=n_jobs,
                                                                                                                                    scripts_dir=osp.join(PRJ_DIR,'Notebooks')))
                                    swarm_file.write('\n')
swarm_file.close()
print("++ INFO: Attempts/Written = [%d/%d]" % (num_entries,num_iters))

++ INFO: Attempts/Written = [35240/52920]
CPU times: user 1.24 s, sys: 1.29 s, total: 2.53 s
Wall time: 13.9 s


> **NOTE**: m=2 all completed (12/06/2022) | m=3 all completed (12/06/2022) | m=5 all completed (12/06/2022) | m=10 all completed (12/06/2022). This is for the whole hyper-parameter space.

## 1.2. Compute Silhouette Index on all scan-level TSNE embeddings

In [10]:
#user specific folders
#=====================
username = getpass.getuser()
print('++ INFO: user working now --> %s' % username)

swarm_folder   = osp.join(PRJ_DIR,'SwarmFiles.{username}'.format(username=username))
logs_folder    = osp.join(PRJ_DIR,'Logs.{username}'.format(username=username))  

swarm_path     = osp.join(swarm_folder,'N08_TSNE_Eval_Clustering_Scans.SWARM.sh')
logdir_path    = osp.join(logs_folder, 'N08_TSNE_Eval_Clustering_Scans.logs')

if not osp.exists(swarm_folder):
    os.makedirs(swarm_folder)
if not osp.exists(logdir_path):
    os.makedirs(logdir_path)
print('++ INFO: Swarm File  : %s' % swarm_path)
print('++ INFO: Logs Folder : %s' % logdir_path)

++ INFO: user working now --> javiergc
++ INFO: Swarm File  : /data/SFIMJGC_HCP7T/manifold_learning_fmri/SwarmFiles.javiergc/N08_TSNE_Eval_Clustering_Scans.SWARM.sh
++ INFO: Logs Folder : /data/SFIMJGC_HCP7T/manifold_learning_fmri/Logs.javiergc/N08_TSNE_Eval_Clustering_Scans.logs


In [15]:
# Open the file
swarm_file = open(swarm_path, "w")
# Log the date and time when the SWARM file is created
swarm_file.write('#Create Time: %s' % datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
swarm_file.write('\n')

# Insert comment line with SWARM command
swarm_file.write('#swarm -J TSNE_10_Scans_SI -f {swarm_path} -b 18 -g 4 -t 4 --time 00:13:00 --partition quick,norm --logdir {logdir_path}'.format(swarm_path=swarm_path,logdir_path=logdir_path))
swarm_file.write('\n')
num_entries = 0 
num_iters = 0

for input_data in input_datas:
    for subject in PNAS2015_subject_list:
        for norm_method in norm_methods:
            for dist in tsne_dist_metrics:
                for init_method in tsne_inits:
                    for pp in tsne_pps:
                        for alpha in tsne_alphas:
                            for m in [10]:                                
                                num_iters += 1
                                input_path  = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',subject,'TSNE',input_data,'{subject}_Craddock_0200.WL{wls}s.WS{wss}s.TSNE_{dist}_pp{pp}_m{m}_a{lr}_{init_method}.{nm}.pkl'.format(subject=subject,
                                                                                                                                                   nm = norm_method,
                                                                                                                                                   wls=str(int(wls)).zfill(3), 
                                                                                                                                                   wss=str(wss),
                                                                                                                                                   init_method=init_method,
                                                                                                                                                   dist=dist,
                                                                                                                                                   pp=str(pp).zfill(4),
                                                                                                                                                   m=str(m).zfill(4),
                                                                                                                                                   lr=str(alpha)))
                                output_path  = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',subject,'TSNE',input_data,'{subject}_Craddock_0200.WL{wls}s.WS{wss}s.TSNE_{dist}_pp{pp}_m{m}_a{lr}_{init_method}.{nm}.SI.pkl'.format(subject=subject,
                                                                                                                                                   nm = norm_method,
                                                                                                                                                   wls=str(int(wls)).zfill(3), 
                                                                                                                                                   wss=str(wss),
                                                                                                                                                   init_method=init_method,
                                                                                                                                                   dist=dist,
                                                                                                                                                   pp=str(pp).zfill(4),
                                                                                                                                                   m=str(m).zfill(4),
                                                                                                                                                   lr=str(alpha)))
                                if not osp.exists(output_path):
                                    num_entries += 1
                                    swarm_file.write('export input={input_path} output={output_path}; sh {scripts_dir}/N10_SI.sh'.format(input_path=input_path, 
                                                                                                                     output_path=output_path,
                                                                                                                     scripts_dir=osp.join(PRJ_DIR,'Notebooks')))
                                    
                                    swarm_file.write('\n')
swarm_file.close()
print("++ INFO: Missing/Needed = [%d/%d]" % (num_entries,num_iters))

++ INFO: Missing/Needed = [0/52920]


> **NOTE:** m=2 (12/06/2022) | m=3 (12/06/2022) | m=5 (12/06/2022) | m=10 (12/06/2022)

***

# 2. Group-level TSNE: "Concatenation + TSNE"

## 2.1 Create Group-level "Concatenation + TSNE" Embeddings

In [19]:
#user specific folders
#=====================
username = getpass.getuser()
print('++ INFO: user working now --> %s' % username)

swarm_folder   = osp.join(PRJ_DIR,'SwarmFiles.{username}'.format(username=username))
logs_folder    = osp.join(PRJ_DIR,'Logs.{username}'.format(username=username))  

swarm_path     = osp.join(swarm_folder,'N08_TSNE_Multitask_ALL.SWARM.sh')
logdir_path    = osp.join(logs_folder, 'N08_TSNE_Multitask_ALL.logs')

if not osp.exists(swarm_folder):
    os.makedirs(swarm_folder)
if not osp.exists(logdir_path):
    os.makedirs(logdir_path)
print('++ INFO: Swarm File  : %s' % swarm_path)
print('++ INFO: Logs Folder : %s' % logdir_path)

++ INFO: user working now --> javiergc
++ INFO: Swarm File  : /data/SFIMJGC_HCP7T/manifold_learning_fmri/SwarmFiles.javiergc/N08_TSNE_Multitask_ALL.SWARM.sh
++ INFO: Logs Folder : /data/SFIMJGC_HCP7T/manifold_learning_fmri/Logs.javiergc/N08_TSNE_Multitask_ALL.logs


In [23]:
%%time
# Open the file
n_jobs=64
swarm_file = open(swarm_path, "w")
# Log the date and time when the SWARM file is created
swarm_file.write('#Create Time: %s' % datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
swarm_file.write('\n')

# Insert comment line with SWARM command
swarm_file.write('#swarm -J TSNE_15_ALL -f {swarm_path} -g 128 -t 64 --partition norm --time 7-00:00:00 --logdir {logdir_path}'.format(swarm_path=swarm_path,logdir_path=logdir_path, n_jobs=n_jobs))
swarm_file.write('\n')
num_entries = 0 
num_iters = 0

for input_data in ['Original']:
    for subject in ['ALL']:
        for norm_method in norm_methods:
            for dist in ['correlation']:
                for init_method in tsne_inits:
                    for pp in tsne_pps:
                        for alpha in [10,1000]:
                            for m in [10]:
                                num_iters += 1
                                path_tvfc = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',subject,input_data,       '{subject}_Craddock_0200.WL{wls}s.WS{wss}s.tvFC.Z.{nm}.pkl'.format(subject=subject,nm=norm_method,wls=str(int(wls)).zfill(3), wss=str(wss)))
                                path_out  = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',subject,'TSNE',input_data,'{subject}_Craddock_0200.WL{wls}s.WS{wss}s.TSNE_{dist}_pp{pp}_m{m}_a{lr}_{init_method}.{nm}.pkl'.format(subject=subject,
                                                                                                                                                   nm = norm_method,
                                                                                                                                                   wls=str(int(wls)).zfill(3), 
                                                                                                                                                   wss=str(wss),
                                                                                                                                                   init_method=init_method,
                                                                                                                                                   dist=dist,
                                                                                                                                                   pp=str(pp).zfill(4),
                                                                                                                                                   m=str(m).zfill(4),
                                                                                                                                                   lr=str(alpha)))
                                if not osp.exists(path_out):
                                    num_entries += 1
                                    swarm_file.write('export path_tvfc={path_tvfc} dist={dist} pp={pp} lr={lr} m={m} n_iter=10000 init={init_method} path_out={path_out} n_jobs={n_jobs} norm={norm_method} grad_method=exact; sh {scripts_dir}/N08_TSNE.sh'.format(path_tvfc=path_tvfc, 
                                                                                                                                    path_out=path_out,
                                                                                                                                    dist=dist,
                                                                                                                                    init_method=init_method,
                                                                                                                                    norm_method=norm_method,
                                                                                                                                    pp=pp,
                                                                                                                                    m=m, 
                                                                                                                                    lr=alpha,
                                                                                                                                    n_jobs=n_jobs,
                                                                                                                                    scripts_dir=osp.join(PRJ_DIR,'Notebooks')))
                                    swarm_file.write('\n')
swarm_file.close()
print("++ INFO: Attempts/Written = [%d/%d]" % (num_entries,num_iters))

++ INFO: Attempts/Written = [3/84]
CPU times: user 5.08 ms, sys: 3.76 ms, total: 8.84 ms
Wall time: 58.7 ms


> **NOTE:** Given computational needs, we only do this part for the following hyper-parameter set: Original data, correlation distance, lr = 10 or 1000, m =2,3,5,10, all norms and all pps

> **NOTE:** m=2 (12/06/2022) | m=3 (12/06/2022) | m=5 (12/06/2022) | m=10 (12/06/2022 - 3 cases missing)

## 2.2. Evaluate Group-level "Concatenation + TSNE" Embeddings

In [33]:
#user specific folders
#=====================
username = getpass.getuser()
print('++ INFO: user working now --> %s' % username)

swarm_folder   = osp.join(PRJ_DIR,'SwarmFiles.{username}'.format(username=username))
logs_folder    = osp.join(PRJ_DIR,'Logs.{username}'.format(username=username))  

swarm_path     = osp.join(swarm_folder,'N12_TSNE_Eval_Clustering_ALL.SWARM.sh')
logdir_path    = osp.join(logs_folder, 'N12_TSNE_Eval_Clustering_ALL.logs')

if not osp.exists(swarm_folder):
    os.makedirs(swarm_folder)
if not osp.exists(logdir_path):
    os.makedirs(logdir_path)
print('++ INFO: Swarm File:  %s' % swarm_path)
print('++ INFO: Logs Folder: %s' % logdir_path)

++ INFO: user working now --> javiergc
++ INFO: Swarm File:  /data/SFIMJGC_HCP7T/manifold_learning_fmri/SwarmFiles.javiergc/N12_TSNE_Eval_Clustering_ALL.SWARM.sh
++ INFO: Logs Folder: /data/SFIMJGC_HCP7T/manifold_learning_fmri/Logs.javiergc/N12_TSNE_Eval_Clustering_ALL.logs


In [35]:
# Open the file
swarm_file = open(swarm_path, "w")
# Log the date and time when the SWARM file is created
swarm_file.write('#Create Time: %s' % datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
swarm_file.write('\n')

# Insert comment line with SWARM command
swarm_file.write('#swarm -J TSNE_ALL_5_SI -f {swarm_path} -b 6 -g 4 -t 4 --time 00:10:00 --partition quick,norm --logdir {logdir_path}'.format(swarm_path=swarm_path,logdir_path=logdir_path))
swarm_file.write('\n')
num_entries = 0 
num_iters = 0

for subject in ['ALL']:
    for input_data in ['Original']:
        for norm_method in norm_methods:
            for dist in ['correlation']:
                for init_method in tsne_inits:
                    for pp in tsne_pps:
                        for alpha in [10,1000]:
                            for m in [5]:
                                num_iters += 1
                                input_path  = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',subject,'TSNE',input_data,'{subject}_Craddock_0200.WL{wls}s.WS{wss}s.TSNE_{dist}_pp{pp}_m{m}_a{lr}_{init_method}.{nm}.pkl'.format(subject=subject,
                                                                                                                                                   nm = norm_method,
                                                                                                                                                   wls=str(int(wls)).zfill(3), 
                                                                                                                                                   wss=str(wss),
                                                                                                                                                   init_method=init_method,
                                                                                                                                                   dist=dist,
                                                                                                                                                   pp=str(pp).zfill(4),
                                                                                                                                                   m=str(m).zfill(4),
                                                                                                                                                   lr=str(alpha)))
                                output_path  = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',subject,'TSNE',input_data,'{subject}_Craddock_0200.WL{wls}s.WS{wss}s.TSNE_{dist}_pp{pp}_m{m}_a{lr}_{init_method}.{nm}.SI.pkl'.format(subject=subject,
                                                                                                                                                   nm = norm_method,
                                                                                                                                                   wls=str(int(wls)).zfill(3), 
                                                                                                                                                   wss=str(wss),
                                                                                                                                                   init_method=init_method,
                                                                                                                                                   dist=dist,
                                                                                                                                                   pp=str(pp).zfill(4),
                                                                                                                                                   m=str(m).zfill(4),
                                                                                                                                                   lr=str(alpha)))
                                #if osp.exists(input_path) & (not osp.exists(output_path)):
                                if (not osp.exists(output_path)):
                                    num_entries += 1
                                    swarm_file.write('export input={input_path} output={output_path}; sh {scripts_dir}/N10_SI.sh'.format(input_path=input_path, 
                                                                                                                     output_path=output_path,
                                                                                                                     scripts_dir=osp.join(PRJ_DIR,'Notebooks')))
                                    
                                    swarm_file.write('\n')
swarm_file.close()
print("++ INFO: Missing/Needed = [%d/%d]" % (num_entries,num_iters))

++ INFO: Missing/Needed = [0/84]


> **NOTE:** m=2 (12/06/2022) | m=3 (12/06/2022) | m=5 (12/06/2022) | m=10 (12/06/2022) --> Missing the same 3 that need to complete

# 2.3 Create and Evalaute group-level TSNE: "TSNE + Procrustes"

In [58]:
#user specific folders
#=====================
username = getpass.getuser()
print('++ INFO: user working now --> %s' % username)

swarm_folder   = osp.join(PRJ_DIR,'SwarmFiles.{username}'.format(username=username))
logs_folder    = osp.join(PRJ_DIR,'Logs.{username}'.format(username=username))  

swarm_path     = osp.join(swarm_folder,'N12_TSNE_Eval_Clustering_Procrustes.SWARM.sh')
logdir_path    = osp.join(logs_folder, 'N12_TSNE_Eval_Clustering_Procrustes.logs')

if not osp.exists(swarm_folder):
    os.makedirs(swarm_folder)
if not osp.exists(logdir_path):
    os.makedirs(logdir_path)
print('++ INFO: Swarm File:  %s' % swarm_path)
print('++ INFO: Logs Folder: %s' % logdir_path)

++ INFO: user working now --> javiergc
++ INFO: Swarm File:  /data/SFIMJGC_HCP7T/manifold_learning_fmri/SwarmFiles.javiergc/N12_TSNE_Eval_Clustering_Procrustes.SWARM.sh
++ INFO: Logs Folder: /data/SFIMJGC_HCP7T/manifold_learning_fmri/Logs.javiergc/N12_TSNE_Eval_Clustering_Procrustes.logs


In [71]:
# Open the file
swarm_file = open(swarm_path, "w")
# Log the date and time when the SWARM file is created
swarm_file.write('#Create Time: %s' % datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
swarm_file.write('\n')

# Insert comment line with SWARM command
swarm_file.write('#swarm -J TSNE_Procrustes_SI -f {swarm_path} -b 14 -g 4 -t 4 --time 00:17:00 --partition quick,norm --logdir {logdir_path}'.format(swarm_path=swarm_path,logdir_path=logdir_path))
swarm_file.write('\n')
num_entries = 0 
num_iters = 0

for input_data in ['Original']:
    for subject in ['Procrustes']:
        for norm_method in norm_methods:
            for dist in ['correlation']:
                for init_method in tsne_inits:
                    for pp in tsne_pps:
                        for alpha in [10,1000]:
                            for m in [2,3,5,10,15,20,25,30]:
                                num_iters += 1
                                emb_path = osp.join(PRJ_DIR,'Data_Interim','PNAS2015','Procrustes','TSNE',input_data,
                                                            'Procrustes_Craddock_0200.WL{wls}s.WS{wss}s.TSNE_{dist}_pp{pp}_m{m}_a{alpha}_{init}.{nm}.pkl'.format(wls=str(int(wls)).zfill(3),alpha=str(alpha),
                                                                                                                      wss=str(wss),init=init_method,
                                                                                                                      dist=dist,
                                                                                                                      pp=str(pp).zfill(4),
                                                                                                                      nm=norm_method,
                                                                                                                      m=str(m).zfill(4)))
                                si_path = osp.join(PRJ_DIR,'Data_Interim','PNAS2015','Procrustes','TSNE',input_data,
                                                           'Procrustes_Craddock_0200.WL{wls}s.WS{wss}s.TSNE_{dist}_pp{pp}_m{m}_a{alpha}_{init}.{nm}.SI.pkl'.format(wls=str(int(wls)).zfill(3), 
                                                                                                                      alpha=str(alpha),init=init_method,
                                                                                                                      wss=str(wss),
                                                                                                                      dist=dist,
                                                                                                                      pp=str(pp).zfill(4),
                                                                                                                      nm=norm_method,
                                                                                                                      m=str(m).zfill(4)))
                                if (not osp.exists(emb_path)) | (not osp.exists(si_path)):
                                    num_entries += 1
                                    swarm_file.write('export sbj_list="{sbj_list}" input_data={input_data} norm_method={norm_method} dist={dist} pp={pp} m={m} alpha={alpha} init_method={init_method} drop_xxxx={drop_xxxx}; sh {scripts_dir}/N11_TSNE_Procrustes.sh'.format(input_data=input_data,
                                                                                                                     sbj_list=','.join(PNAS2015_subject_list),
                                                                                                                     norm_method=norm_method,
                                                                                                                     dist=dist,
                                                                                                                     pp=str(pp),
                                                                                                                     m=str(m),
                                                                                                                     alpha=str(alpha),
                                                                                                                     init_method = init_method,
                                                                                                                     drop_xxxx = 'False',
                                                                                                                     scripts_dir=osp.join(PRJ_DIR,'Notebooks')))
                                    
                                    swarm_file.write('\n')
swarm_file.close()
print("++ INFO: Missing/Needed = [%d/%d]" % (num_entries,num_iters))

++ INFO: Missing/Needed = [0/672]


***
# Load and save into a single dataframe

In [50]:
from utils.io import load_TSNE_SI

In [ ]:
%%time
RELOAD_SI_TSNE = True
if RELOAD_SI_TSNE:
    print('++ INFO: Loading SI for Concat + TSNE....')
    si_TSNE_all              = load_TSNE_SI(sbj_list=['ALL'],               check_availability=False, verbose=True, wls=wls, wss=wss, ms=[2,3,5,10], dist_metrics=['correlation'], input_datas=['Original'], alphas=[10,1000])
    print('++ INFO: Loading SI for TSNE + Procrustes....')
    si_TSNE_procrustes       = load_TSNE_SI(sbj_list=['Procrustes'],        check_availability=False, verbose=True, wls=wls, wss=wss, ms=[2,3,5,10,15,20,25,30], dist_metrics=['correlation'], input_datas=['Original'], alphas=[10,1000])
    print('++ INFO: Loading SI for scan level TSNE...')
    si_TSNE_scans            = load_TSNE_SI(sbj_list=PNAS2015_subject_list, check_availability=False, verbose=True, wls=wls, wss=wss, ms=[2,3,5,10])

    si_TSNE = pd.concat([si_TSNE_all, si_TSNE_procrustes])
    si_TSNE = pd.concat([si_TSNE_scans, si_TSNE_all, si_TSNE_procrustes])

    si_TSNE.replace('Window Name','Task', inplace=True)
    si_TSNE = si_TSNE.set_index(['Subject','Input Data','Norm','Metric','PP','m','Alpha','Init','Target']).sort_index()
    del si_TSNE_scans, si_TSNE_all, si_TSNE_procrustes, si_TSNE_procrustes_extra, si_TSNE_scans_extra
    
    si_TSNE.to_pickle(osp.join(PRJ_DIR,'Dashboard','Data','si_TSNE.pkl'))
else:
    si_TSNE = pd.read_pickle(osp.join(PRJ_DIR,'Dashboard','Data','si_TSNE.pkl'))

++ INFO: Loading SI for Concat + TSNE....


Subjects::   0%|          | 0/1 [00:00<?, ?it/s]

Data Inputs::   0%|          | 0/1 [00:00<?, ?it/s]

-e "/data/SFIMJGC_HCP7T/manifold_learning_fmri/Data_Interim/PNAS2015/ALL/TSNE/Original/ALL_Craddock_0200.WL045s.WS1.5s.TSNE_correlation_pp0065_m0010_a10_pca.zscored.SI.pkl" -e "/data/SFIMJGC_HCP7T/manifold_learning_fmri/Data_Interim/PNAS2015/ALL/TSNE/Original/ALL_Craddock_0200.WL045s.WS1.5s.TSNE_correlation_pp0090_m0010_a10_pca.zscored.SI.pkl" -e "/data/SFIMJGC_HCP7T/manifold_learning_fmri/Data_Interim/PNAS2015/ALL/TSNE/Original/ALL_Craddock_0200.WL045s.WS1.5s.TSNE_correlation_pp0150_m0010_a10_pca.zscored.SI.pkl" ++ INFO [load_TSNE_SI]: Number of files missing = [3/336] files
++ INFO: Loading SI for TSNE + Procrustes....


Subjects::   0%|          | 0/1 [00:00<?, ?it/s]

...

Data Inputs::   0%|          | 0/3 [00:00<?, ?it/s]

Data Inputs::   0%|          | 0/3 [00:00<?, ?it/s]

Data Inputs::   0%|          | 0/3 [00:00<?, ?it/s]

Data Inputs::   0%|          | 0/3 [00:00<?, ?it/s]

Data Inputs::   0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
si_TSNE = pd.concat([si_TSNE_scans, si_TSNE_all, si_TSNE_procrustes_extra])
si_TSNE.replace('Window Name','Task', inplace=True)
si_TSNE = si_TSNE.set_index(['Subject','Input Data','Norm','Metric','PP','m','Alpha','Init','Target']).sort_index()

In [55]:
si_TSNE.to_pickle(osp.join(PRJ_DIR,'Dashboard','Data','si_TSNE.pkl'))

In [56]:
si_TSNE

SI
Subject Input Data Norm    Metric      PP  m  Alpha Init Target           
ALL     Original   asis    correlation 20  2  10    pca  Subject -0.138877
                                                         Task    -0.042321
                                              1000  pca  Subject -0.121037
                                                         Task    -0.043349
                                           3  10    pca  Subject -0.104175
...                                                                    ...
SBJ27   Original   zscored euclidean   200 10 75    pca  Task     0.225440
                                              100   pca  Task     0.225456
                                              200   pca  Task     0.225381
                                              500   pca  Task     0.224185
                                              1000  pca  Task     0.222926

[213018 rows x 1 columns]

***
***
# END OF NOTEBOOK
***
***

### Extra cases for the Classification Study

In [7]:
%%time
RELOAD_SI_TSNE = False
if RELOAD_SI_TSNE:
    si_TSNE_all        = load_TSNE_SI(sbj_list=['ALL'],               check_availability=False, verbose=False, wls=wls, wss=wss, ms=[2,3])
    si_TSNE_procrustes = load_TSNE_SI(sbj_list=['Procrustes'],        check_availability=False, verbose=False, wls=wls, wss=wss, ms=[2,3])
    si_TSNE_scans      = load_TSNE_SI(sbj_list=PNAS2015_subject_list, check_availability=False, verbose=False, wls=wls, wss=wss, ms=[2,3])
    
    si_TSNE = pd.concat([si_TSNE_scans, si_TSNE_all, si_TSNE_procrustes])
    si_TSNE.replace('Window Name','Task', inplace=True)
    si_TSNE = si_TSNE.set_index(['Subject','Input Data','Norm','Metric','PP','m','Alpha','Init','Target']).sort_index()
    del si_TSNE_scans, si_TSNE_all, si_TSNE_procrustes
    
    si_TSNE.to_pickle(osp.join(PRJ_DIR,'Dashboard','Data','si_TSNE.pkl'))
else:
    si_TSNE = pd.read_pickle(osp.join(PRJ_DIR,'Dashboard','Data','si_TSNE.pkl'))

CPU times: user 6.44 ms, sys: 5.43 ms, total: 11.9 ms
Wall time: 40.5 ms


In [16]:
_,tsne_best_norm_method,tsne_best_dist, tsne_best_pp, _, tsne_best_alpha,_,_ = si_TSNE.loc[PNAS2015_subject_list,'Original',:,:,:,:,:,:,'Task'].to_xarray().mean(dim='Subject').to_dataframe().sort_values(by='SI',ascending=False).iloc[0].name
print('++ INFO: Best scan-level configuration --> NM=%s, DIST=%s, PP=%d, ALPHA=%f' % (tsne_best_norm_method,tsne_best_dist, tsne_best_pp, tsne_best_alpha))

++ INFO: Best scan-level configuration --> NM=asis, DIST=correlation, PP=70, ALPHA=10.000000
